In [1]:
import pandas as pd
import os

# Decomposition by component and heating system

#### Loading data

In [2]:
stock_buildings = pd.read_csv('../../output/building_stock_sdes2018_aggregated.csv', index_col=[0, 1, 2, 3, 4, 5]).squeeze()

In [3]:
idx = stock_buildings.xs('Multi-family', level='Housing type', drop_level=False).xs('Oil fuel', level='Heating energy', drop_level=False).index
stock_buildings.drop(idx, inplace=True)

idx = stock_buildings.xs('Multi-family', level='Housing type', drop_level=False).xs('Wood fuel', level='Heating energy', drop_level=False).index
stock_buildings.drop(idx, inplace=True)

In [4]:
archetypes = pd.read_csv(os.path.join('output', 'archetypes.csv'))
archetypes.rename(columns={'U_wall': 'Wall', 'U_floor': 'Floor', 'U_roof': 'Roof', 'U_windows': 'Windows', 'Proba': 'Share', 'Performance': 'Energy performance'}, inplace=True)
archetypes['Energy performance'].replace({'A': 'B'}, inplace=True)
archetypes.set_index(['Energy performance', 'Housing type', 'Heating energy'], inplace=True)
archetypes.drop(['Electricity', 'Number'], inplace=True, axis=1)
archetypes

Wall  Floor  Roof  Windows  \
Energy performance Housing type  Heating energy                               
B                  Multi-family  Natural gas      0.2    0.0   0.0      1.3   
                   Single-family Natural gas      0.2    0.3   0.1      1.3   
                                 Oil fuel         0.2    0.3   0.1      1.3   
                                 Wood fuel        0.2    0.3   0.1      1.3   
C                  Multi-family  Natural gas      0.4    0.0   0.0      2.2   
...                                               ...    ...   ...      ...   
G                  Multi-family  Electricity      1.9    0.0   0.0      6.2   
                                 Electricity      2.5    0.0   0.0      3.3   
                   Single-family Electricity      1.7    0.5   0.1      6.8   
                                 Electricity      0.6    2.1   0.3      5.6   
                                 Electricity      1.6    2.3   0.3     13.7   

                                                 Efficiency     Share  
Energy performance Housing type  Heating energy                        
B                  Multi-family  Natural gas           0.85  1.000000  
                   Single-family Natural gas           0.85  1.000000  
                                 Oil fuel              0.85  1.000000  
                                 Wood fuel             0.85  1.000000  
C                  Multi-family  Natural gas           0.75  1.000000  
...                                                     ...       ...  
G                  Multi-family  Electricity           0.95  0.646274  
                                 Electricity           0.95  0.353726  
                   Single-family Electricity           0.95  0.432415  
                                 Electricity           0.95  0.409536  
                                 Electricity           0.95  0.158049  

[81 rows x 6 columns]

In [5]:
stock_buildings  = stock_buildings.reset_index('Energy performance').replace({'A': 'B'}).set_index('Energy performance', append=True)

In [6]:
new_levels = ['Wall', 'Floor', 'Roof', 'Windows', 'Efficiency']
new_stock = pd.DataFrame()
for idx, group_stock in stock_buildings.groupby(['Energy performance', 'Housing type', 'Heating energy']):
    for i, row in archetypes.loc[idx, :].iterrows():
        stock = row['Share'] * group_stock
        stock = pd.concat((stock, pd.concat([row[new_levels]] * group_stock.shape[0], keys=group_stock.index, axis=1).T), axis=1).set_index(new_levels, append=True)
        new_stock = pd.concat((new_stock, stock), axis=0)

/Users/lucas/opt/anaconda3/envs/DataAnalysis/lib/python3.8/site-packages/pandas/core/indexing.py:925: PerformanceWarning: indexing past lexsort depth may impact performance.
  return self._getitem_tuple(key)


In [7]:
new_stock['Heating system'] = new_stock.index.get_level_values('Efficiency')
new_stock['Heating system'].replace({0.9: 'Performance boiler', 0.65: 'Standard boiler', 2: 'Heat pump'}, inplace=True)
new_stock.set_index('Heating system', append=True, inplace=True)
new_stock = new_stock.droplevel(['Energy performance', 'Efficiency'])
new_stock = new_stock.groupby(new_stock.index.names).sum()
print(new_stock)


                                                                                                                 Stock buildings
Housing type  Heating energy Occupancy status Income owner Income tenant Wall Floor Roof Windows Heating system                 
Multi-family  Electricity    Owner-occupied   D1           D1            0.2  0.0   0.0  1.3     3.00                2225.100088
                                                                         0.3  0.0   0.0  4.5     0.95                5688.251234
                                                                         0.5  0.0   0.0  2.4     0.95               10388.296466
                                                                         0.6  0.0   0.0  6.7     0.95               15750.031490
                                                                         1.2  0.0   0.0  2.9     3.00                5649.117553
...                                                                                              

In [8]:
print('\n Total number of housings {:,.0f}'.format(new_stock.squeeze().sum() ))


 Total number of housings 26,147,807


In [9]:
new_stock = new_stock.reset_index(level=['Heating energy', 'Heating system'])
new_stock['Heating system'].replace({3: 'Heat pump'}, inplace=True)
new_stock['Heating system'].replace({0.95: 'Performance boiler'}, inplace=True)
new_stock['Heating system'].replace({0.85: 'Performance boiler'}, inplace=True)
new_stock['Heating system'].replace({0.75: 'Standard boiler'}, inplace=True)

new_stock['Heating system'] = new_stock['Heating energy'] + '-' + new_stock['Heating system']
new_stock = new_stock.set_index(['Heating system'], append=True).loc[:, 'Stock buildings']

In [10]:
new_stock.to_csv(os.path.join('output', 'building_stock_sdes2018.csv'))